https://github.com/4GeeksAcademy/Hector-machine-learning-algorithm-python-template.git

https://github.com/4GeeksAcademy/Hector-Logistic-Regression.git

In [1]:
# Librerias
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from pickle import dump

In [2]:
#Importamos los datos a un data frame
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv", sep = ";")
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
#Eliminamos los duplicados
df = df.drop_duplicates().reset_index(drop = True)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
#Comprobamos los valores nulos
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41176 entries, 0 to 41175
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   job             41176 non-null  object 
 2   marital         41176 non-null  object 
 3   education       41176 non-null  object 
 4   default         41176 non-null  object 
 5   housing         41176 non-null  object 
 6   loan            41176 non-null  object 
 7   contact         41176 non-null  object 
 8   month           41176 non-null  object 
 9   day_of_week     41176 non-null  object 
 10  duration        41176 non-null  int64  
 11  campaign        41176 non-null  int64  
 12  pdays           41176 non-null  int64  
 13  previous        41176 non-null  int64  
 14  poutcome        41176 non-null  object 
 15  emp.var.rate    41176 non-null  float64
 16  cons.price.idx  41176 non-null  float64
 17  cons.conf.idx   41176 non-null 

In [6]:
#Sustituimos los valores objeto por valores factorizados
# Crear un diccionario para almacenar un LabelEncoder para cada columna categórica
le_dict = {}

# Identificar las columnas categóricas
categorical_cols = ["job", "marital","education", "default","housing", "loan","contact", "month", "day_of_week","poutcome", "y"]

# Aplicar LabelEncoder a cada columna categórica
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le  # Guardar el encoder en el diccionario

df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,3,1,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,3,0,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,1,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,3,0,0,2,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [7]:
#Seleccionamos los datos de entrenamiento y test
X = df.drop("y", axis = 1)
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [8]:
X_train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
12727,44,3,1,3,0,2,2,0,3,1,118,1,999,0,1,1.4,93.918,-42.7,4.960,5228.1
35799,41,10,1,5,0,2,0,0,6,1,34,7,999,0,1,-1.8,92.893,-46.2,1.244,5099.1
1423,44,6,1,5,0,0,0,1,6,2,626,6,999,0,1,1.1,93.994,-36.4,4.855,5191.0
52,60,0,1,3,0,0,0,1,6,1,253,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0
28739,26,8,2,6,0,2,0,0,0,2,76,4,999,0,1,-1.8,93.075,-47.1,1.410,5099.1


In [9]:
X_test.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
36254,59,5,1,5,0,2,2,0,4,1,1460,1,999,0,1,-2.9,92.963,-40.8,1.266,5076.2
32662,34,1,1,2,0,0,0,0,6,1,248,1,999,1,0,-1.8,92.893,-46.2,1.299,5099.1
40860,32,9,1,5,0,2,0,0,8,4,275,2,999,0,1,-1.1,94.601,-49.5,0.959,4963.6
32897,53,0,1,3,0,2,2,0,6,1,186,2,999,1,0,-1.8,92.893,-46.2,1.299,5099.1
16461,24,7,2,2,0,2,0,1,3,4,184,1,999,0,1,1.4,93.918,-42.7,4.963,5228.1


In [10]:
#Entrenamos el modelo
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [11]:
#Realizamos una prediccion con el modelo creado y los datos de test
y_pred = model.predict(X_test)
y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [12]:
#Comprobamos la precision del modelo
accuracy_score(y_test, y_pred)

0.9082078678970374

In [13]:
#Optimizamo el modelo mediante hiperparametros
hyperparams = {
    'C': uniform(0.01, 100),
    'penalty': ['l2'],
    'solver': ['liblinear']
}

random_search = RandomizedSearchCV(model, hyperparams, n_iter=20, scoring='accuracy', cv=5, n_jobs=-1)
random_search

RandomizedSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x70fd50ce7550>,
                                        'penalty': ['l2'],
                                        'solver': ['liblinear']},
                   scoring='accuracy')

In [14]:
#Evitamos los warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Ajusta el modelo con RandomizedSearchCV
random_search.fit(X_train, y_train)

# Imprime los mejores parámetros y la mejor puntuación
print(f"Mejores parámetros: {random_search.best_params_}")
print(f"Mejor puntuación: {random_search.best_score_}")


Mejores parámetros: {'C': np.float64(78.59225906363244), 'penalty': 'l2', 'solver': 'liblinear'}
Mejor puntuación: 0.9100329304399615


In [15]:
#Reentrenamos el nuevo modelo con los hyperparametros

model_op = LogisticRegression(C = 54.350987006057125, penalty = "l2", solver = "saga")
model_op.fit(X_train, y_train)

LogisticRegression(C=54.350987006057125, solver='saga')

In [16]:
y_pred_op = model_op.predict(X_test)
y_pred_op

array([1, 0, 0, ..., 0, 0, 0])

In [17]:
#Comprobamos la nueva pecision del modelo optimizado 
accuracy_score(y_test, y_pred_op)

0.9060223409422049

Tarda mucho en ecnontrar los hiperparametros mas adecuados, por lo tanot no he encontrado una optipización del modelo por lo tanto el 1º modelo puede vale como optimo ya que tine una precision del 90.8% 